In [486]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from copy import deepcopy

In [487]:
scheduler_df = pd.read_csv("./scheduler.csv")
pd.DataFrame(scheduler_df)
scheduler_df

,Program Name,PM,Start Date,End Date
0,Program 1,Person 1,9/20/2025,9/23/2025
1,Program 2,Person 2,9/15/2025,9/15/2025
2,Program 3,Person 2,9/10/2025,9/20/2025
3,Program 4,Person 2,9/5/2025,9/6/2025
4,Program 5,Person 3,9/5/2025,9/5/2025
5,Program 6,Person 1,9/10/2025,9/11/2025
6,Program 7,Person 1,9/5/2025,9/12/2025
7,Program 8,Person 2,9/5/2025,9/5/2025


In [488]:
def calculate_available_pms_for_week(week_start, scheduler_df):
    week_end = week_start + timedelta(days=4)  # Assuming a week goes from Monday to Friday
    busy_pms = set()

    for _, program in scheduler_df.iterrows():
        if not (program['End Date'] < week_start or program['Start Date'] > week_end):
            busy_pms.add(program['PM'])

    total_pms = scheduler_df['PM'].nunique()
    available_pms = total_pms - len(busy_pms)

    #print(week_start, available_pms)
    return available_pms

In [489]:
# Assuming scheduler_df is the dataframe containing the program information
# with 'Start Date' and 'End Date' columns as datetime objects

# Assuming 'Start Date' and 'End Date' are in 'MM/DD/YYYY' format
scheduler_df['Start Date'] = pd.to_datetime(scheduler_df['Start Date'], format='%m/%d/%Y')
scheduler_df['End Date'].fillna(scheduler_df['Start Date'], inplace=True)
scheduler_df['End Date'] = pd.to_datetime(scheduler_df['End Date'], format='%m/%d/%Y')

# Find the earliest start date and the latest end date
earliest_start = scheduler_df['Start Date'].min()
latest_end = scheduler_df['End Date'].max()

# Find the Monday before the earliest start date
days_until_monday = (earliest_start.weekday() + 1) % 7
first_monday = earliest_start - timedelta(days=days_until_monday)

# Find the Friday after the latest end date
days_until_friday = (4 - latest_end.weekday()) % 7
last_friday = latest_end + timedelta(days=days_until_friday)

# Calculate the total number of weeks
total_weeks = (last_friday - first_monday).days // 7 + 1

# Get the total unique PMs
total_unique_pms = scheduler_df['PM'].nunique()

# Initialize the matrix
hours_matrix = np.full((total_weeks, 5), 7 * total_unique_pms)

for week_index in range(total_weeks):
    week_start = first_monday + timedelta(weeks=week_index)
    available_pms = calculate_available_pms_for_week(week_start, scheduler_df)
    hours_matrix[week_index, :] = 7 * available_pms * 2  # 7 hours per day, multiplied by 2 as per your requirement

# Define the function to update the matrix for each program
def update_hours_matrix(matrix, start_date, end_date, first_monday):
    # Calculate program duration in days and initial hours to subtract based on your criteria
    program_duration = (end_date - start_date).days + 1
    hours_to_subtract = program_duration * 2 * 7  # Adjust based on your specific rules
    week_index = (start_date - first_monday).days // 7
    day_index = min(start_date.weekday(), 4)  # Get the weekday index for the start date

    day_count = 0

    while hours_to_subtract > 0 and week_index >= 0:
    
        # Calculate the subtraction amount for this iteration
        if day_count < 6:
            subtraction_amount = 5  # Starting week special rule
        elif 6 <= day_count <= 10:
            subtraction_amount = 2  # Second week before the program starts
        else:
            subtraction_amount = 1  # From the third week before and onwards
        #print("week,day index", week_index, day_index, hours_to_subtract)
        # Ensure we don't subtract more than what's available or needed
        actual_subtraction = min(subtraction_amount, hours_to_subtract)
        if week_index >= 0 and day_index >= 0:  # Check bounds
            matrix[week_index, day_index] -= actual_subtraction
            hours_to_subtract -= actual_subtraction

        # Move to the previous day, wrapping to the previous week if necessary
        if day_index == 0:
            week_index -= 1
            day_index = 4  # Wrap to Friday of the previous week
        else:
            day_index -= 1  # Move to the previous day within the same week
        day_count += 1
    
# Update the matrix for each program
for index, program in scheduler_df.iterrows():
    update_hours_matrix(hours_matrix, 
                        program['Start Date'], 
                        program['End Date'], 
                        first_monday)

# Convert the matrix to a DataFrame for better readability
hours_df = pd.DataFrame(hours_matrix, 
                        index=pd.date_range(first_monday, periods=total_weeks, freq='W-MON'),
                        columns=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])

hours_df

,Monday,Tuesday,Wednesday,Thursday,Friday
2025-09-01,29,29,15,11,10
2025-09-08,2,2,2,8,4
2025-09-15,18,23,23,23,23
2025-09-22,28,28,28,28,28


In [490]:
def update_hours_matrix2(matrix, start_date, end_date, first_monday):
    new_matrix = deepcopy(matrix)
    # Calculate program duration in days and initial hours to subtract based on your criteria
    program_duration = (end_date - start_date).days + 1
    hours_to_subtract = program_duration * 2 * 7  # Adjust based on your specific rules
    week_index = (start_date - first_monday).days // 7
    day_index = min(start_date.weekday(), 4)  # Get the weekday index for the start date
  
   # print("start_date", start_date, hours_to_subtract, week_index)
    day_count = 0

    while hours_to_subtract > 0 and week_index >= 0:
    
        # Calculate the subtraction amount for this iteration
        if day_count < 6:
            subtraction_amount = 5  # Starting week special rule
        elif 6 <= day_count <= 10:
            subtraction_amount = 2  # Second week before the program starts
        else:
            subtraction_amount = 1  # From the third week before and onwards
    
        # Ensure we don't subtract more than what's available or needed
        actual_subtraction = min(subtraction_amount, hours_to_subtract)
        if week_index >= 0 and day_index >= 0:  # Check bounds
            new_matrix[week_index, day_index] -= actual_subtraction
            hours_to_subtract -= actual_subtraction

        # Move to the previous day, wrapping to the previous week if necessary
        if day_index == 0:
            week_index -= 1
            day_index = 4  # Wrap to Friday of the previous week
        else:
            day_index -= 1  # Move to the previous day within the same week
        
        day_count += 1
    return new_matrix

In [491]:
PROPOSED_DATE = "9/18/2025"
PROGRAM_DURATION_DAYS = 3



In [492]:
program_start_date = pd.to_datetime(PROPOSED_DATE)

original_hours_matrix = deepcopy(hours_matrix)
print(original_hours_matrix, "\n")

# -- 

matrix = update_hours_matrix2(original_hours_matrix, program_start_date - timedelta(weeks=1), program_start_date - timedelta(weeks=1)+ timedelta(days=PROGRAM_DURATION_DAYS - 1), first_monday)
print(matrix)
before_week_index = ((program_start_date - timedelta(weeks=1)) - first_monday).days // 7
print("before week index", before_week_index)
print("sum row", sum(matrix[before_week_index]))
print(" ")
# ----

matrix = update_hours_matrix2(original_hours_matrix, program_start_date, program_start_date + timedelta(days=PROGRAM_DURATION_DAYS - 1), first_monday)
print(matrix)
during_week_index = (program_start_date - first_monday).days // 7
print("during week index", during_week_index)
print("sum row", sum(matrix[during_week_index]))
print(" ")

#--- 

matrix = update_hours_matrix2(original_hours_matrix, program_start_date + timedelta(weeks=1), program_start_date + timedelta(weeks=1, days=PROGRAM_DURATION_DAYS - 1), first_monday)
print(matrix)
after_week_index = ((program_start_date + timedelta(weeks=1)) - first_monday).days // 7
print("after week index", after_week_index)
print("sum row", sum(matrix[after_week_index]))
print(" ")

# ---


[[29 29 15 11 10]
 [ 2  2  2  8  4]
 [18 23 23 23 23]
 [28 28 28 28 28]] 

[[27 27 13  6  5]
 [-3 -3 -3  3  4]
 [18 23 23 23 23]
 [28 28 28 28 28]]
before week index 1
sum row -2
 
[[29 28 14  9  8]
 [ 0  0  0  3 -1]
 [13 18 18 18 23]
 [28 28 28 28 28]]
during week index 2
sum row 90
 
[[29 29 15 11 10]
 [ 2  1  1  6  2]
 [16 21 21 18 18]
 [23 23 23 23 28]]
after week index 3
sum row 120
 
